# Search good hyper-parameters in Normalization
- It seems finding a good value helps a lot in performance

In [ ]:
import sys

from urllib3.http2.probe import acquire_and_get

sys.path.append("../Share")
import config, Processing_same_with_MATLAB, Feature_norm_parms
import os
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

feature_names = ['Zero Crossing (ZC)', 'Slope Sign Changes (SSC)', 'Waveform Length (WL)', 'WAMP', 'Mean Absolute Value (MAV)', 'Mean Square (MS)', 'Root Mean Square (RMS)',
                 'v-order 3 (V3)', 'log detector (LD)', 'difference absolute standard deviation value (DASDV)', 'maximum fractal length (MFL)', 'myopulse percentage rate (MPR)',
                 'mean absolute value slope (MAVS)', 'weighted mean absolute (WMS)',
                 'Cepstrum Coefficient 1', 'Cepstrum Coefficient 2', 'Cepstrum Coefficient 3', 'Cepstrum Coefficient Average', 'DWTC1', 'DWTC2',
                 'DWTPC1', 'DWTPC2', 'DWTPC3']

fs = round(10e6 / 2048)  # 4883 Hz
lower_cutoff = 100
upper_cutoff = 600
filter_b, filter_a = Processing_same_with_MATLAB.cheby2(4, 30, [lower_cutoff / (fs/2), upper_cutoff / (fs/2)], btype='bandpass')

# 14 features


In [ ]:
import Train_and_test_one_feature_with_norm_param

Trainer_feature_wise = Train_and_test_one_feature_with_norm_param.main(filter_a,
                                                          filter_b,
                                                          data_files = config.dataset_sub_C,
                                                          default_path = config.default_path_sub_C,
                                                          SUBJECT = "Carlson",
                                                          K = 1) #K : K 이전 데이터 학습, K이후 데이터 테스트

In [ ]:
Normalization_TF = True
feature_idx = 2  # WL
mean_list = [1.5, 2, 2.5]
std_list = [0.8, 0.9, 1, 1.2]  #std_list = [0.5, 0.65, 0.8]
All_acc, All_param = [], []

for M in mean_list:
    for S in std_list:
        test_acc, X_train_stacked, y_train_stacked, X_test_stacked, y_test_stacked = Trainer_feature_wise.Train_and_test(
            Normalization_TF, M, S, num_feature_set=14, target_feat_idx=feature_idx
        )
        All_acc.append(np.mean(test_acc))
        All_param.append([M, S])

# Find best param
best_idx = np.argmax(All_acc)
best_param = All_param[best_idx]
best_acc = All_acc[best_idx]

In [ ]:
print(f"Best Accuracy: {best_acc:.4f} with mean={best_param[0]}, std={best_param[1]}")

df = pd.DataFrame(All_param, columns=["Mean", "Std"])
df["Accuracy"] = All_acc
print(df.pivot(index="Mean", columns="Std", values="Accuracy"))

In [ ]:
print(f"Best Accuracy: {best_acc:.4f} with mean={best_param[0]}, std={best_param[1]}")

df2 = pd.DataFrame(All_param, columns=["Mean", "Std"])
df2["Accuracy"] = All_acc
print(df2.pivot(index="Mean", columns="Std", values="Accuracy"))

In [ ]:
test_acc #Without Norm

In [ ]:
test_acc #With Norm

In [ ]:
이제 좋은 param 찾기

In [ ]:
근데 다 같이 했을때 Normalization이 중요한거고, 다 같이 안하면 크게 상관없는 거 아닌가요
- 아니지, 일단 좋은 param을 찾고 그거를 합치면 더 좋아질듯